In [244]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("test") \
    .getOrCreate()

In [245]:
df= spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

25/06/26 16:21:00 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [246]:
from pyspark.sql.functions import explode, split

In [247]:
words = df.select(explode(split(df.value, " ")).alias("word"))

In [248]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [249]:
output_schema = StructType([
    StructField("word", StringType()),
    StructField("count", IntegerType())
])

# Define state schema (just count as int)
state_schema = StructType([
    StructField("count", IntegerType())
])

In [250]:
import pandas as pd

In [251]:
# checking what happens at timeout what are the parameters
# what after key is deleted from statestore
# and what after removing will it allow the key to generate again or not
# if so it shouldnot use its prevoius state right

In [252]:
def update_state(key,value_iterator,state):
    values=list(value_iterator)
    # yo value_iterator vanae ko pani generator class ko raixa
    # so yek taali iterate gare paxi aarko choti ta kaa milxa ta aani
    # aajhai pani dataframe of values aaudo raixa so
    #values=[[dataframe]]
    with open("/opt/spark_streaming/keyvale1.txt","a") as file:
        file.write(f"1. key is {key} \n")
        file.write(f"2. value is {values[0]['word']} \n")
        file.write(f"3. timeout happened={state.hasTimedOut}\n")

        if state.hasTimedOut:
            state.remove()
            c=0
        else:
            c=len(values[0]['word'])
            b=state.get if state.exists else 0
            c=b+c
            state.setTimeoutDuration(60 * 1000)
            state.update(pd.Series([c]))
        pdf = pd.DataFrame({
            "word": key,
            "count": c
        })

        file.write(f" 4. final_c = {c} \n")

    # Return an iterator of one dataframe
    yield pdf

In [253]:
stateful_df = words.groupBy("word").applyInPandasWithState(
    update_state,
    outputStructType=output_schema,
    stateStructType=state_schema,
    outputMode="update",
    timeoutConf="ProcessingTimeTimeout"
)


In [254]:
query = stateful_df.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()

25/06/26 16:21:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-96881e60-7641-44b5-b35b-93864c22f758. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/26 16:21:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    6|
|aaditya|    2|
|     hi|    6|
+-------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



...

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 41
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 42
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



[Stage 87:===========>                                           (40 + 8) / 200]

# so this proves everything

1.after timeout 

empty value_list for that key

sate.hastimedout is true


2.after doing state.remove it will remove that key and its value from the store


3.after deleting it obviously cannot use previous value as it is deleted

and major question was state store ma rakhnai na dini ho ki k ho

haina raixa rakhna dini but give it a fresh start vane ko previous values use garni chai haina

naya aauda jasto hunthyo tei approach lagu hunxa